https://sambaiga.github.io/2017/05/03/hmm-intro.html
https://hmmlearn.readthedocs.io/en/latest/tutorial.html#
1、Given the model parameters and observed data, estimate the optimal sequence of hidden states.
2、Given the model parameters and observed data, calculate the model likelihood.
3、Given just the observed data, estimate the model parameters.
    The first and the second problem can be solved by the dynamic programming algorithms known as the Viterbi algorithm and the Forward-Backward algorithm, respectively. The last one can be solved by an iterative Expectation-Maximization (EM) algorithm, known as the Baum-Welch algorithm.
    
https://github.com/tostq/Easy_HMM  https://www.cnblogs.com/fangbei/p/8409110.html

https://www.jb51.net/article/137068.htm

In [5]:
# Building HMM and generating samples
import numpy as np
from hmmlearn import hmm
np.random.seed(42)
model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.6, 0.3, 0.1])
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
print(np.tile(np.identity(2), (3, 1, 1)))
X, Z = model.sample(10)
print(X)
print(Z)
print(model)

[[[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]]
[[-1.11188012  0.31890218]
 [ 1.57921282  0.76743473]
 [-0.58087813 -0.52516981]
 [ 0.24196227 -1.91328024]
 [ 0.81644508 -1.523876  ]
 [-0.90802408 -1.4123037 ]
 [-0.62947496  0.59772047]
 [-0.54438272  0.11092259]
 [-0.60025385  0.94743982]
 [-0.60170661  1.85227818]]
[0 0 0 0 0 0 0 0 0 0]
GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=3, n_iter=10, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)


https://www.cnblogs.com/pinard/p/7001397.html
https://www.cnblogs.com/pinard/p/6945257.html

    hmmlearn实现了三种HMM模型类，按照观测状态是连续状态还是离散状态，可以分为两类。GaussianHMM和GMMHMM是连续观测状态的HMM模型，而MultinomialHMM是离散观测状态的模型，也是我们在HMM原理系列篇里面使用的模型。

    对于MultinomialHMM的模型，使用比较简单，"startprob_"参数对应我们的隐藏状态初始分布ΠΠ, "transmat_"对应我们的状态转移矩阵AA, "emissionprob_"对应我们的观测状态概率矩阵BB。
    
    对于连续观测状态的HMM模型，GaussianHMM类假设观测状态符合高斯分布，而GMMHMM类则假设观测状态符合混合高斯分布。一般情况下我们使用GaussianHMM即高斯分布的观测状态即可。以下对于连续观测状态的HMM模型，我们只讨论GaussianHMM类。
　　
      在GaussianHMM类中，"startprob_"参数对应我们的隐藏状态初始分布ΠΠ, "transmat_"对应我们的状态转移矩阵AA, 比较特殊的是观测状态概率的表示方法，此时由于观测状态是连续值，我们无法像MultinomialHMM一样直接给出矩阵BB。而是采用给出各个隐藏状态对应的观测状态高斯分布的概率密度函数的参数。
　　　
       如果观测序列是一维的，则观测状态的概率密度函数是一维的普通高斯分布。如果观测序列是NN维的，则隐藏状态对应的观测状态的概率密度函数是NN维高斯分布。高斯分布的概率密度函数参数可以用μμ表示高斯分布的期望向量，ΣΣ表示高斯分布的协方差矩阵。在GaussianHMM类中，“means”用来表示各个隐藏状态对应的高斯分布期望向量μμ形成的矩阵，而“covars”用来表示各个隐藏状态对应的高斯分布协方差矩阵ΣΣ形成的三维张量。

In [2]:
# 建立HMM的模型
import numpy as np
from hmmlearn import hmm

states = ["box 1", "box 2", "box3"]
n_states = len(states)

observations = ["red", "white"]
n_observations = len(observations)

# 隐藏状态初始分布 从第一个盒子抽球的概率是0.2，从第二个盒子抽球的概率是0.4，从第三个盒子抽球的概率是0.4
start_probability = np.array([0.2, 0.4, 0.4])

# 状态转移矩阵 规则是：如果当前抽球的盒子是第一个盒子，则以0.5的概率仍然留在第一个盒子继续抽球，以0.2的
# 概率去第二个盒子抽球，以0.3的概率去第三个盒子抽球。如果当前抽球的盒子是第二个盒子，则以0.5的概率仍然
# 留在第二个盒子继续抽球，以0.3的概率去第一个盒子抽球，以0.2的概率去第三个盒子抽球。如果当前抽球的盒子
# 是第三个盒子，则以0.5的概率仍然留在第三个盒子继续抽球，以0.2的概率去第一个盒子抽球，以0.3的概率去第二个盒子抽球

transition_probability = np.array([
  [0.5, 0.2, 0.3],
  [0.3, 0.5, 0.2],
  [0.2, 0.3, 0.5]
])

# 观测状态概率矩阵
emission_probability = np.array([
  [0.5, 0.5],
  [0.4, 0.6],
  [0.7, 0.3]
])

model = hmm.MultinomialHMM(n_components=n_states)
model.startprob_=start_probability
model.transmat_=transition_probability

model.emissionprob_=emission_probability

In [11]:
# 根据Observ预测 observ score
# init state * trans --> box1 prob, box2 prob, box3 prob  * ems_prob --> box1 prob * white prob + box2 prob + ....
# print(model.emissionprob_)
box_t = np.dot(model.startprob_, model.transmat_)
# print(box_t)
obser_t = np.dot(box_t, model.emissionprob_)
# print(obser_t)
seen = np.array([[0,1,0,1]]).T

print(seen.shape)
# model.score Compute the log probability under the model.
print('observ score -->', np.exp(model.score(seen))) 
#  predict proba 算法 ？？？？  Compute the posterior probability for each state in the model.
print('predict prob --->', model.predict_proba(seen))

first = model.predict_proba(seen)
s_box_t = np.dot(first, model.transmat_)
s_obser_t = np.dot(s_box_t, model.emissionprob_)

print('second observ ->', s_obser_t)
seen2 = np.array([[0,0]]).T
print('second observ prob -->{}, first observ prob -->{}'.format(np.exp(model.score(seen2)),np.exp(model.score(seen))))
print(0.291999/0.54)
# 根据以上，可以根据观察值通过predict_proba得到最后的状态概率分布情况，然后再通过np.dot 转移和发散概率得到下一个观察值的概率
# 或通过model score来计算， model.score([x0,x1]) / model.score([x0])来得到下一状态为x1的概率


(4, 1)
observ score --> 0.06009079999999997
predict prob ---> [[0.18715344 0.32414147 0.48870509]
 [0.3153511  0.42476519 0.25988371]
 [0.32051828 0.30139589 0.37808583]
 [0.35076751 0.41917365 0.23005884]]
second observ -> [[0.53845408 0.46154592]
 [0.52655825 0.47344175]
 [0.53627278 0.46372722]
 [0.52594308 0.47405692]]
second observ prob -->0.2919999999999999, first observ prob -->0.06009079999999997
0.5407388888888889


In [12]:
seen = np.array([[1,0,0]]).T
logprob, box = model.decode(seen, algorithm="viterbi")

print("The ball picked:", ", ".join(map(lambda x: observations[x], seen.T[0])))
print("The hidden box", ", ".join(map(lambda x: states[x], box)))
print(logprob)

box2 = model.predict(seen)
print("The ball picked:", ", ".join(map(lambda x: observations[x], seen.T[0])))
print("The hidden box", ", ".join(map(lambda x: states[x], box2)))

# 要注意的是score函数返回的是以自然对数为底的对数概率值，我们在HMM问题一中手动计算的结果是未取对数的原始概率是0.13022。对比一下
print(model.score(seen))

The ball picked: white, red, red
The hidden box box3, box3, box3
-4.219907785197447
The ball picked: white, red, red
The hidden box box3, box3, box3
-2.0481134318001732


In [13]:
# HMM问题二，求解模型参数的问题。由于鲍姆-韦尔奇算法是基于EM算法的近似算法，所以
# 我们需要多跑几次，比如下面我们跑三次，选择一个比较优的模型参数
import numpy as np
from hmmlearn import hmm

states = ["box 1", "box 2", "box3"]
n_states = len(states)

observations = ["red", "white"]
n_observations = len(observations)
model2 = hmm.MultinomialHMM(n_components=n_states, n_iter=20, tol=0.01)
X2 = np.array([[0,1,0,1],[0,0,0,1],[1,0,1,1]])
model2.fit(X2)
print('start prob -->',model2.startprob_)
print('trans mat -->', model2.transmat_)
print('emiss prob -->', model2.emissionprob_)
print('score -->', model2.score(X2))
model2.fit(X2)
print('start prob -->',model2.startprob_)
print('trans mat -->', model2.transmat_)
print('emiss prob -->', model2.emissionprob_)
print('score -->', model2.score(X2))
model2.fit(X2)
print('start prob -->',model2.startprob_)
print('trans mat -->', model2.transmat_)
print('emiss prob -->', model2.emissionprob_)
print('score -->', model2.score(X2))


start prob --> [1.74812358e-04 9.99825188e-01 1.12572606e-28]
trans mat --> [[4.90132251e-01 1.69643411e-01 3.40224338e-01]
 [3.92010029e-02 4.64996378e-05 9.60752497e-01]
 [2.38315058e-01 3.82282522e-01 3.79402420e-01]]
emiss prob --> [[9.01144713e-01 9.88552875e-02]
 [9.99672801e-01 3.27198543e-04]
 [4.88437063e-02 9.51156294e-01]]
score --> -6.699264973458398
start prob --> [5.83760939e-05 1.86540686e-21 9.99941624e-01]
trans mat --> [[5.03149425e-01 3.64551119e-01 1.32299456e-01]
 [1.97819100e-01 3.50804119e-01 4.51376781e-01]
 [3.68410448e-02 9.63142817e-01 1.61379603e-05]]
emiss prob --> [[7.79122191e-01 2.20877809e-01]
 [1.00569597e-01 8.99430403e-01]
 [9.99590853e-01 4.09146873e-04]]
score --> -6.6816696113774
start prob --> [1.86429858e-28 9.97944672e-01 2.05532825e-03]
trans mat --> [[0.35802433 0.37742245 0.26455323]
 [0.92154437 0.00248437 0.07597125]
 [0.33840819 0.19988858 0.46170323]]
emiss prob --> [[0.04391321 0.95608679]
 [0.99583563 0.00416437]
 [0.84212442 0.1578755

In [14]:
# GaussianHMM实例
# 观测状态是二维的，而隐藏状态有4个。因此我们的“means”参数是4×2的矩阵，而“covars”协方差参数是4×2×2的张量
startprob = np.array([0.6, 0.3, 0.1, 0.0])
# The transition matrix, note that there are no transitions possible
# between component 1 and 3
transmat = np.array([[0.7, 0.2, 0.0, 0.1],
                     [0.3, 0.5, 0.2, 0.0],
                     [0.0, 0.3, 0.5, 0.2],
                     [0.2, 0.0, 0.2, 0.6]])
# The means of each component
means = np.array([[0.0,  0.0],
                  [0.0, 11.0],
                  [9.0, 10.0],
                  [11.0, -1.0]])
# The covariance of each component
covars = .5 * np.tile(np.identity(2), (4, 1, 1))

# Build an HMM instance and set parameters
model3 = hmm.GaussianHMM(n_components=4, covariance_type="full")

# Instead of fitting it from the data, we directly set the estimated
# parameters, the means and covariance of the components
model3.startprob_ = startprob
model3.transmat_ = transmat
model3.means_ = means
model3.covars_ = covars

seen = np.array([[1.1,2.0],[-1,2.0],[3,7]])
logprob, state = model3.decode(seen, algorithm="viterbi")
print(state)
# 对数概率
print(model3.score(seen))


[0 0 1]
-41.121128137687
